In [1]:
%matplotlib inline
import subprocess
import sys
import numpy as np
import pandas as pd
import pylab as plt
from collections import namedtuple


In [2]:
cols = "allocated, reclaimed, highwater, curr_allocated, b_allocate_clocks, b_available_blocks, b_unused_pools, b_lost_pool_headers, b_lost_quantization, b_lost_arena_alignment, total".split(', ')

In [3]:
def get_meminfo():
    p = subprocess.Popen([sys.executable, "-c", "import sys; sys._debugmallocstats()"],
                     stderr=subprocess.PIPE)
    s = p.stderr.read().decode("utf-8").split(sep='\n')
    shift = 4
    table = np.c_[[np.array(s[n_class + shift].split()).astype(np.int)
              for n_class in range(62)]]
    
    i_alloc = shift + len(table) + 1
    df_blocks = pd.DataFrame(table, columns=["class", "size", "n_pools", "use_blocks", "avail_blocks"]).set_index('class')
    df = pd.DataFrame([[float(s[i].split()[-1].replace(',', '')) for i in list(range(i_alloc, i_alloc+4))+list(range(i_alloc+6, i_alloc+13))]], columns=cols)
    return df_blocks, df 

In [4]:
import benchpy as bp
from examples.bp_samples import cycle_list, html_parse

In [5]:
from io import StringIO
import os 

data = StringIO(open(
        os.path.join("examples/data",
                     "html5lib_spec.html")).read())
f_html = bp.bench(html_parse, data).f

In [6]:
batch_sizes = range(1, 100, 10)

In [7]:
# f = bp.bench(cycle_list, 100).f
f = f_html

In [8]:
df_blocks_start,_ = get_meminfo()

In [9]:
res = []
df = pd.DataFrame(columns=cols)
for batch in batch_sizes:
    t = bp.get_time_perf_counter(f, batch)
    _df_blocks, _df = get_meminfo()
    df = df.append(_df, ignore_index=True)
    res.append([batch, t])
res = np.array(res)    

In [10]:
pd.DataFrame(res, columns=["batch", "time"]).merge(df, left_index=True, right_index=True, how='outer')

,batch,time,allocated,reclaimed,highwater,curr_allocated,b_allocate_clocks,b_available_blocks,b_unused_pools,b_lost_pool_headers,b_lost_quantization,b_lost_arena_alignment,total
0,1,63.963441,8,2,6,6,1274776,116368,147456,16704,17560,0,1572864
1,11,0.057733,8,2,6,6,1274776,116368,147456,16704,17560,0,1572864
2,21,0.169940,8,2,6,6,1274776,116368,147456,16704,17560,0,1572864
3,31,0.315352,8,2,6,6,1274776,116368,147456,16704,17560,0,1572864
4,41,0.365955,8,2,6,6,1274776,116368,147456,16704,17560,0,1572864
5,51,0.533924,8,2,6,6,1274776,116368,147456,16704,17560,0,1572864
6,61,0.594900,8,2,6,6,1274776,116368,147456,16704,17560,0,1572864
7,71,0.861418,8,2,6,6,1274776,116368,147456,16704,17560,0,1572864
8,81,0.901249,8,2,6,6,1274776,116368,147456,16704,17560,0,1572864
9,91,1.154293,8,2,6,6,1274776,116368,147456,16704,17560,0,1572864


In [11]:
(_df_blocks - df_blocks_start).sum()

size            0
n_pools         0
use_blocks      0
avail_blocks    0
dtype: int64

In [ ]:
_df_blocks.plot